In [ ]:
import csv
import os
import pickle
from datetime import datetime as dtm
from time import sleep

from igmapper.components.followers import getUserFollowers
from igmapper.components.following import getUserFollowing
from igmapper.components.profile import getUserProfileInfo
from igmapper.components.utils import bold, italic
from igmapper.lib.webbrowser import extract_cookies, start_browser

# Define paths for source and logs
CWD = os.getcwd()  # current working directory
SRC, LOG = f"{CWD}/src", f"{CWD}/logs"
PKL = f"{SRC}/cookies.pkl"

CHROMEDRIVER = f"{CWD}/chromedriver"
URL = "https://instagram.com/"
XPATH_CONTAINER_FEED = "//*[contains(@class, 'xw7yly9')]"


def save_to_csv(file_name, data, headers):
    """Salva dados em um arquivo CSV."""
    with open(file_name, "w", encoding="utf-8", newline="") as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(headers)
        csv_writer.writerows(data)


print(f"{bold('Instagram Followers & Unfollowers')}\n")

usr = input(bold("Enter the Instagram username (without '@'): "))
if not usr:
    print("No username provided. Exiting.")
    exit(1)

following_csv = f"{LOG}/{usr}_following.csv"
followers_csv = f"{LOG}/{usr}_followers.csv"
non_followers_csv = f"{LOG}/{usr}_non_followers.csv"
cookie_valid, retry = False, 1

while cookie_valid == False and retry <= 3:
    retry += 1
    try:
        cookie = pickle.load(open(PKL, "rb"))
        print("Cookie file founded...")

        for i in cookie:
            if "csrftoken" in i["name"]:
                csrftoken = i["value"]
            if "sessionid" in i["name"]:
                sessionid, expiry = i["value"], i["expiry"]

        expiry, now = dtm.fromtimestamp(expiry), dtm.now()
        if now > expiry:
            raise
        cookie_valid = True
    except:
        print(f"{bold('Login • Instagram')}\n{italic('You need to login to continue')}")
        driver = start_browser(URL, CHROMEDRIVER, False, True)
        sleep(60)
        extract_cookies(driver, SRC)
# Retrieve user account information

print("Retrieving user account ID...")
acc = getUserProfileInfo(usr, csrftoken)

# If the account information could not be retrieved, exit the program
if not acc:
    print("Failed to retrieve account information. Exiting.")
    exit(1)
else:
    # Save account profile information to a JSON log file
    open(f"{LOG}/{usr}-profile_info.json", "w").write(str(acc))

    # Extract user ID from the fetched account data
    user = acc["data"]["user"]
    user_id = user["id"]

bio = user["biography"]
bio_links = user["bio_links"]
category_name = user["category_name"]
follow = user["edge_follow"]["count"]
followed = user["edge_followed_by"]["count"]
full_name = user["full_name"]
is_private = user["is_private"]
is_verified = user["is_verified"]
posts_count = user["edge_owner_to_timeline_media"]["count"]

# {italic(f'https://www.instagram.com/{usr}')}
output = f"""{bold('Investigated profile:')}
－ {usr} {"🟓" if is_verified else ""}
－ {bold(posts_count)} posts     {bold(followed)} followers     {bold(follow)} following
－ {full_name if full_name else "No name"}
－ {italic(category_name) if category_name else "No category"}
－ "{bio if bio else "No bio"}"
－ {f'{[i["url"] for i in bio_links]}' if bio_links else "No links"}
－ {bold('This account is private') if is_private else "Public account"}"""
print(output)
# Retrieve the user's followers and following lists using the user ID
try:
    print("Get followers...")
    acc_followers = getUserFollowers(user_id, csrftoken, sessionid)
    print("Get following...")
    acc_following = getUserFollowing(user_id, csrftoken, sessionid)
except Exception as e:
    exit()
# Create dictionaries to store 'following' and 'followers' details
following_dict, followers_dict = {}, {}

# Populate the 'following' dictionary with usernames and full names
print("Building following dictionary...")
for i in acc_following:
    usr = i.get("username")
    following_dict[usr] = i.get("full_name")

# Populate the 'followers' dictionary with usernames and full names
print("Building followers dictionary...")
for i in acc_followers:
    usr = i.get("username")
    followers_dict[usr] = i.get("full_name")

# Find users that are in 'following' but not in 'followers' (people who don't follow back)
print("Get non followers...")
non_followers = [i for i in following_dict if i not in followers_dict]
csv_columns = ["Index", "Full Name", "Username", "Profile URL"]

# 1. "Não seguidores" (quem você segue, mas não te segue de volta)
if non_followers:
    if input(bold("Save non followers list? [Y/n]: ")).strip().lower() in ("", "y"):
        data = []
        for i, usr in enumerate(sorted(non_followers), 1):
            fullname, profile_url = following_dict[usr], f"{URL}{usr}"
            data.append([i, fullname, usr, profile_url])

        # Salvar no CSV
        save_to_csv(non_followers_csv, data, csv_columns)

if input(bold("Save following list? [Y/n]: ")).strip().lower() in ("", "y"):
    data = [
        [i, usr["full_name"], usr["username"], f"{URL}{usr['username']}"]
        for i, usr in enumerate(acc_following, 1)
    ]
    save_to_csv(following_csv, data, csv_columns)

# 3. Lista de "seguidores"
if input(bold("Save followers list? [Y/n]: ")).strip().lower() in ("", "y"):
    data = [
        [i, usr["full_name"], usr["username"], f"{URL}{usr['username']}"]
        for i, usr in enumerate(acc_followers, 1)
    ]
    save_to_csv(followers_csv, data, csv_columns)